# Generativne mreže

Rekurentne nevronske mreže (RNN) in njihove različice z vrati, kot so celice Long Short Term Memory (LSTM) in Gated Recurrent Units (GRU), omogočajo modeliranje jezika, tj. lahko se naučijo vrstnega reda besed in napovedujejo naslednjo besedo v zaporedju. To nam omogoča uporabo RNN za **generativne naloge**, kot so običajno generiranje besedila, strojno prevajanje in celo opisovanje slik.

V arhitekturi RNN, ki smo jo obravnavali v prejšnji enoti, je vsaka enota RNN ustvarila naslednje skrito stanje kot izhod. Vendar pa lahko vsaki rekurentni enoti dodamo še en izhod, kar nam omogoča, da ustvarimo **zaporedje** (ki je enako dolžini izvirnega zaporedja). Poleg tega lahko uporabimo RNN enote, ki ne sprejemajo vhoda na vsakem koraku, ampak le začetni vektorski stanji, nato pa ustvarijo zaporedje izhodov.

V tem zvezku se bomo osredotočili na preproste generativne modele, ki nam pomagajo ustvarjati besedilo. Za enostavnost bomo zgradili **mrežo na ravni znakov**, ki generira besedilo črko za črko. Med učenjem moramo vzeti nek korpus besedila in ga razdeliti na zaporedja črk.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Gradnja besedišča znakov

Za izgradnjo generativne mreže na ravni znakov moramo besedilo razdeliti na posamezne znake namesto na besede. Plast `TextVectorization`, ki smo jo uporabljali prej, tega ne omogoča, zato imamo dve možnosti:

* Ročno naložimo besedilo in izvedemo tokenizacijo "na roke", kot je prikazano v [tem uradnem primeru Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Uporabimo razred `Tokenizer` za tokenizacijo na ravni znakov.

Odločili se bomo za drugo možnost. `Tokenizer` se lahko uporablja tudi za tokenizacijo na ravni besed, zato je prehod z tokenizacije na ravni znakov na tokenizacijo na ravni besed precej preprost.

Za izvedbo tokenizacije na ravni znakov moramo podati parameter `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Prav tako želimo uporabiti en poseben znak za označevanje **konca zaporedja**, ki ga bomo imenovali `<eos>`. Dodajmo ga ročno v besednjak:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Zdaj lahko za kodiranje besedila v zaporedja števil uporabimo:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Učenje generativnega RNN za generiranje naslovov

Način, kako bomo učili RNN za generiranje novičarskih naslovov, je naslednji. Na vsakem koraku bomo vzeli en naslov, ki bo podan v RNN, in za vsak vhodni znak bomo od mreže zahtevali, da generira naslednji izhodni znak:

![Slika, ki prikazuje primer generiranja besede 'HELLO' z RNN.](../../../../../translated_images/sl/rnn-generate.56c54afb52f9781d.webp)

Za zadnji znak našega zaporedja bomo od mreže zahtevali, da generira `<eos>` token.

Glavna razlika med generativnim RNN, ki ga uporabljamo tukaj, je v tem, da bomo vzeli izhod iz vsakega koraka RNN, ne le iz zadnje celice. To lahko dosežemo z nastavitvijo parametra `return_sequences` za RNN celico.

Tako bo med učenjem vhod v mrežo zaporedje kodiranih znakov določene dolžine, izhod pa bo zaporedje iste dolžine, vendar premaknjeno za en element in zaključeno z `<eos>`. Minibatch bo sestavljen iz več takšnih zaporedij, pri čemer bomo morali uporabiti **polnjenje** (padding), da uskladimo vsa zaporedja.

Ustvarimo funkcije, ki bodo za nas preoblikovale podatkovni niz. Ker želimo zaporedja polniti na ravni minibatch-a, bomo najprej razvrstili podatkovni niz z uporabo `.batch()`, nato pa ga `map`-ali, da izvedemo preoblikovanje. Tako bo funkcija za preoblikovanje kot parameter sprejela celoten minibatch:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Nekaj pomembnih stvari, ki jih tukaj počnemo:
* Najprej izluščimo dejansko besedilo iz niza tensorja
* `text_to_sequences` pretvori seznam nizov v seznam celoštevilskih tensorjev
* `pad_sequences` dopolni te tensorje do njihove največje dolžine
* Na koncu vse znake kodiramo v enovrstično kodiranje, prav tako izvedemo premik in dodamo `<eos>`. Kmalu bomo videli, zakaj potrebujemo znake v enovrstičnem kodiranju.

Vendar pa je ta funkcija **Pythonic**, kar pomeni, da je ni mogoče samodejno prevesti v Tensorflow računski graf. Če poskušamo to funkcijo neposredno uporabiti v funkciji `Dataset.map`, bomo dobili napake. To Pythonic klic moramo oviti z uporabo ovojnice `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Opomba**: Razlikovanje med Pythonovimi in Tensorflow funkcijami za transformacijo podatkov se morda zdi nekoliko zapleteno, in morda se sprašujete, zakaj ne transformiramo nabora podatkov z uporabo standardnih Pythonovih funkcij, preden ga posredujemo funkciji `fit`. Čeprav je to vsekakor mogoče, ima uporaba `Dataset.map` veliko prednost, saj se cevovod za transformacijo podatkov izvaja z uporabo Tensorflow računalniškega grafa, kar omogoča izkoriščanje GPU izračunov in zmanjšuje potrebo po prenašanju podatkov med CPU in GPU.

Zdaj lahko zgradimo našo generativno mrežo in začnemo z učenjem. Temeljiti mora na katerikoli rekurentni celici, o kateri smo govorili v prejšnji enoti (enostavna, LSTM ali GRU). V našem primeru bomo uporabili LSTM.

Ker mreža prejme znake kot vhod, velikost besedišča pa je precej majhna, ne potrebujemo vgraditvene plasti (embedding layer), saj lahko enovročno kodiran vhod neposredno vstopi v LSTM celico. Izhodna plast bo `Dense` klasifikator, ki bo pretvoril izhod LSTM v enovročno kodirane številke tokenov.

Poleg tega, ker delamo z zaporedji spremenljive dolžine, lahko uporabimo plast `Masking`, da ustvarimo masko, ki bo ignorirala dopolnjene dele niza. To sicer ni strogo potrebno, saj nas ne zanima preveč vse, kar presega token `<eos>`, vendar jo bomo uporabili, da pridobimo nekaj izkušenj s to vrsto plasti. `input_shape` bo `(None, vocab_size)`, kjer `None` označuje zaporedje spremenljive dolžine, izhodna oblika pa je prav tako `(None, vocab_size)`, kot lahko vidite iz `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generiranje izhoda

Zdaj, ko smo model izurili, ga želimo uporabiti za generiranje izhoda. Najprej potrebujemo način za dekodiranje besedila, predstavljenega kot zaporedje številk žetonov. Za to bi lahko uporabili funkcijo `tokenizer.sequences_to_texts`; vendar ta ne deluje dobro pri žetonizaciji na ravni znakov. Zato bomo vzeli slovar žetonov iz žetonizatorja (imenovan `word_index`), zgradili obratni zemljevid in napisali svojo funkcijo za dekodiranje:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Zdaj bomo začeli z generacijo. Začeli bomo z neko začetno vrednostjo `start`, jo zakodirali v zaporedje `inp`, nato pa bomo v vsakem koraku poklicali našo mrežo, da izračuna naslednji znak.

Izhod mreže `out` je vektor z `vocab_size` elementi, ki predstavljajo verjetnosti za vsak token. Najbolj verjetno številko tokena lahko najdemo z uporabo `argmax`. Ta znak nato dodamo v seznam generiranih tokenov in nadaljujemo z generacijo. Ta postopek generiranja enega znaka ponovimo `size`-krat, da ustvarimo želeno število znakov, pri čemer pa se postopek predčasno zaključi, če naletimo na `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Vzorčenje izhoda med učenjem

Ker nimamo nobenih uporabnih metrik, kot je *natančnost*, je edini način, da vidimo, ali se naš model izboljšuje, **vzorčenje** generiranih nizov med učenjem. Za to bomo uporabili **povratne klice**, tj. funkcije, ki jih lahko podamo funkciji `fit` in ki se bodo občasno klicale med učenjem.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ta primer že ustvarja precej dober tekst, vendar ga je mogoče izboljšati na več načinov:

* **Več besedila**. Uporabili smo samo naslove za našo nalogo, vendar bi morda želeli eksperimentirati s celotnim besedilom. Ne pozabite, da RNN-ji niso najboljši pri obdelavi dolgih zaporedij, zato je smiselno, da jih razdelite na krajše stavke ali pa vedno trenirate na fiksni dolžini zaporedja z vnaprej določeno vrednostjo `num_chars` (recimo 256). Poskusite zgornji primer spremeniti v takšno arhitekturo, pri čemer uporabite [uradni Kerasov vodič](https://keras.io/examples/generative/lstm_character_level_text_generation/) kot navdih.

* **Večplastni LSTM**. Smiselno je poskusiti z 2 ali 3 plastmi LSTM celic. Kot smo omenili v prejšnji enoti, vsaka plast LSTM izlušči določene vzorce iz besedila, in v primeru generatorja na ravni znakov lahko pričakujemo, da bo nižja raven LSTM odgovorna za izluščanje zlogov, višje ravni pa za besede in besedne zveze. To je mogoče preprosto implementirati z dodajanjem parametra za število plasti v konstruktor LSTM.

* Prav tako bi morda želeli eksperimentirati z **GRU enotami** in preveriti, katere delujejo bolje, ter z **različnimi velikostmi skritih plasti**. Prevelika skrita plast lahko povzroči prekomerno prileganje (npr. mreža se bo naučila točno določenega besedila), premajhna velikost pa morda ne bo dala dobrih rezultatov.


## Generiranje mehkejšega besedila in temperatura

V prejšnji definiciji funkcije `generate` smo vedno izbrali znak z najvišjo verjetnostjo kot naslednji znak v generiranem besedilu. To je pogosto povzročilo, da se je besedilo "vrtelo" med istimi zaporedji znakov znova in znova, kot v tem primeru:
```
today of the second the company and a second the company ...
```

Če pa pogledamo porazdelitev verjetnosti za naslednji znak, lahko opazimo, da razlika med nekaj najvišjimi verjetnostmi ni velika, npr. en znak ima verjetnost 0.2, drugi pa 0.19 itd. Na primer, ko iščemo naslednji znak v zaporedju '*play*', je naslednji znak lahko enako verjetno presledek ali **e** (kot v besedi *player*).

To nas pripelje do zaključka, da ni vedno "pravično" izbrati znak z višjo verjetnostjo, saj lahko izbira drugega najverjetnejšega znaka še vedno vodi do smiselnega besedila. Bolj smiselno je **vzorec** znakov vzeti iz porazdelitve verjetnosti, ki jo poda izhod mreže.

To vzorčenje lahko izvedemo z uporabo funkcije `np.multinomial`, ki implementira tako imenovano **multinomsko porazdelitev**. Funkcija, ki implementira to **mehko** generiranje besedila, je definirana spodaj:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Predstavili smo še en parameter, imenovan **temperatura**, ki se uporablja za označevanje, kako strogo se moramo držati najvišje verjetnosti. Če je temperatura 1,0, izvajamo pošteno multinomno vzorčenje, in ko temperatura gre proti neskončnosti - vse verjetnosti postanejo enake, naključno izberemo naslednji znak. V spodnjem primeru lahko opazimo, da besedilo postane nesmiselno, ko preveč povečamo temperaturo, in spominja na "ciklirano" težko generirano besedilo, ko se približa 0.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
